In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, plot_confusion_matrix, confusion_matrix, plot_roc_curve, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
import xgboost as xgb
import warnings
import seaborn as sn
pd.set_option('display.max_columns', None)

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../../data/UsableData.csv")
holdoutdf = pd.read_csv("../../data/UntestableData.csv")
df = df.drop(columns=['Unnamed: 0'])
df['HvarType'] = df['HvarType'].mask(df['HvarType'] != 'C', other='V')

df = df.drop(columns=['Catalog', 'VarFlag', '---', 'Period', 'moreVar', 'morePhoto', 'Source'])
holdoutdf = holdoutdf.drop(columns=['Unnamed: 0', 'Catalog', 'VarFlag', '---', 'Period', 'moreVar', 'morePhoto', 'Source'])

mask = {
    'C': 0,
    'V': 1
}
df['Target'] = df['HvarType'].map(mask)
# df['AstroRefIndicator'] = df['AstroRef'].isna()
# mask = {
#     True: 1,
#     False: 0
# }
# df['AstroRefIndicator'] = df['AstroRefIndicator'].map(mask)

x = df.drop(columns=['HvarType','Target','HIP'])
y = df['Target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=42)

In [3]:
def displayMetrics(fitModel, xtr, xte, ytr, yte):
    train_preds = fitModel.predict(xtr)
    test_preds = fitModel.predict(xte)
    train_probs = fitModel.predict_proba(xtr)[:,1]
    test_probs = fitModel.predict_proba(xte)[:,1]

    train_accuracy = accuracy_score(ytr, train_preds)
    test_accuracy = accuracy_score(yte, test_preds)
    train_recall = recall_score(ytr, train_preds, zero_division=0)
    test_recall = recall_score(yte, test_preds, zero_division=0)
    train_precision = precision_score(ytr, train_preds, zero_division=0)
    test_precision = precision_score(yte, test_preds, zero_division=0)
    train_f1 = f1_score(ytr, train_preds, zero_division=0)
    test_f1 = f1_score(yte, test_preds, zero_division=0)
    train_rocauc = roc_auc_score(ytr, train_probs)
    test_rocauc = roc_auc_score(yte, test_probs)

    print(F'\nTrain Accuracy:\t\t{train_accuracy}')
    print(F'Test Accuracy:\t\t{test_accuracy}')
    print(F"\nTrain Recall:\t\t{train_recall}")
    print(F'Test Recall:\t\t{test_recall}')
    print(F"\nTrain Precision:\t{train_precision}")
    print(F'Test Precision:\t\t{test_precision}')
    print(F"\nTrain f1:\t\t{train_f1}")
    print(F'Test f1:\t\t{test_f1}')
    print(F"\nTrain ROC-AUC:\t\t{train_rocauc}")
    print(F'Test ROC-AUC:\t\t{test_rocauc}')
    
    plot_confusion_matrix(fitModel, xte, yte)

In [4]:
numCols = [col for col in df.columns if df[col].dtype != 'O']
objCols = [col for col in df.columns if df[col].dtype == 'O']


In [5]:
# fun = []
# fun = numCols

In [29]:
# fun.remove("RAdeg")
# fun.remove("HD")

In [31]:
# fun.remove("BTmag")

In [33]:
# fun.remove("VTmag")

In [35]:
# fun.remove("e_VTmag")

In [37]:
# fun.remove("Hpmag")

In [38]:
# fun.remove("Hpmax")

In [40]:
# fun.remove("HPmin")

In [42]:
# fun.remove("e_DEdeg")

In [44]:
# fun.remove("e_pmDE")

In [46]:
# fun.remove("e_pmRA")

In [48]:
# fun.remove("pmDE:pmRA")

In [50]:
# fun.remove("pmDE:RA")

In [52]:
# fun.remove("(V-I)red")

In [54]:
# fun.remove("V-I")

In [56]:
# fun.remove("Hpscat")

In [58]:
# fun.remove("Ncomp")

In [6]:
# df[fun].corr()

,HIP,Vmag,RAdeg,DEdeg,Plx,pmRA,pmDE,e_RAdeg,e_DEdeg,e_Plx,e_pmRA,e_pmDE,DE:RA,Plx:RA,Plx:DE,pmRA:RA,pmRA:DE,pmRA:Plx,pmDE:RA,pmDE:DE,pmDE:Plx,pmDE:pmRA,F1,F2,BTmag,e_BTmag,VTmag,e_VTmag,B-V,e_B-V,V-I,e_V-I,Hpmag,e_Hpmag,Hpscat,o_Hpmag,Hpmax,HPmin,Nsys,Ncomp,theta,rho,e_rho,dHp,e_dHp,HD,(V-I)red,Target
HIP,1.000000,0.005603,0.999867,-0.014275,-0.001337,-0.028091,-0.009449,0.006865,0.006921,0.003868,0.008539,0.001978,0.020286,-0.499665,0.051221,0.083660,0.050860,0.042807,0.070914,0.025945,0.079432,-0.055308,-0.003102,-0.015720,0.013923,0.017451,0.005711,0.008180,0.024390,0.011522,0.026753,0.013885,0.005427,0.010639,0.013848,0.040626,0.003434,0.006048,0.001649,0.004370,-0.005534,0.023507,-0.005192,-0.001144,-0.014720,0.858588,0.027701,0.015230
Vmag,0.005603,1.000000,0.005074,-0.006714,-0.076837,0.000787,-0.036742,0.197477,0.198944,0.302839,0.252194,0.268502,0.002481,-0.026535,-0.018517,-0.040343,-0.007377,0.008040,-0.005967,-0.019249,-0.010638,0.001178,0.017501,0.022283,0.885359,0.513846,0.996395,0.704777,0.078055,0.214042,0.086106,0.242594,0.997618,0.191309,0.181560,-0.053876,0.992203,0.994848,0.173799,-0.052645,-0.009034,0.021242,0.234420,-0.133417,0.146736,0.115833,0.082092,-0.066025
RAdeg,0.999867,0.005074,1.000000,-0.014474,-0.001299,-0.028862,-0.009414,0.006954,0.006961,0.003973,0.008592,0.002126,0.019341,-0.499887,0.050313,0.084921,0.050449,0.042106,0.070176,0.027313,0.077654,-0.055905,-0.003021,-0.015525,0.013672,0.017524,0.005533,0.008312,0.024061,0.012100,0.026416,0.013899,0.004942,0.010572,0.013854,0.039571,0.003063,0.005510,0.000754,0.004397,-0.005582,0.023437,-0.005092,-0.001151,-0.014671,0.858687,0.027448,0.015191
DEdeg,-0.014275,-0.006714,-0.014474,1.000000,-0.007203,0.006984,-0.011244,-0.012804,-0.016224,-0.016633,-0.015155,-0.018135,-0.020387,0.274880,-0.039435,-0.011347,0.037640,-0.062354,0.055442,0.009235,0.053619,0.040145,-0.008704,0.009211,-0.008348,0.007690,-0.004245,0.003405,-0.009318,0.014375,-0.005959,0.029969,-0.007505,-0.010118,-0.004455,-0.016047,-0.005360,-0.005663,0.040002,-0.008080,0.009668,-0.017719,-0.005409,0.015768,0.013637,0.042362,-0.001573,-0.012115
Plx,-0.001337,-0.076837,-0.001299,-0.007203,1.000000,-0.035570,-0.229038,0.085810,0.082538,0.121926,0.118566,0.118937,0.006584,-0.016958,-0.024114,-0.013747,-0.004115,0.007648,-0.005188,0.001706,-0.014590,0.003736,0.065030,0.031809,-0.138464,0.011398,-0.140990,-0.018317,0.015849,-0.041107,0.034465,-0.026624,-0.072840,0.027667,-0.052869,-0.028384,-0.076788,-0.082672,0.069408,0.031163,0.033456,0.031817,-0.010440,-0.036962,-0.053929,-0.009221,0.044271,0.000448
pmRA,-0.028091,0.000787,-0.028862,0.006984,-0.035570,1.000000,-0.052740,0.005190,0.003574,0.011809,0.004464,0.004450,0.093028,0.052483,-0.005985,-0.017664,0.018266,0.054382,0.023102,-0.009652,0.005030,0.087166,-0.004160,-0.004206,-0.005161,-0.007273,-0.008271,-0.006535,0.005965,0.001009,0.004355,-0.002492,0.001118,0.002375,-0.000033,0.001469,-0.002240,-0.001888,-0.039176,0.002778,-0.016377,-0.000114,0.005728,0.011538,-0.005189,-0.000809,0.003518,-0.002769
pmDE,-0.009449,-0.036742,-0.009414,-0.011244,-0.229038,-0.052740,1.000000,0.014549,0.018125,0.000602,-0.001800,0.007854,-0.003750,0.006033,0.043334,0.007203,0.008727,-0.011292,0.007562,0.001426,0.006941,-0.007498,-0.007070,-0.012132,-0.021979,-0.040457,-0.013146,-0.045528,-0.044090,0.013796,-0.048293,0.005163,-0.038832,0.030946,0.009532,0.061282,-0.045085,-0.042726,0.019363,0.006599,0.006081,-0.009875,-0.019426,0.029597,0.006856,-0.028386,-0.048470,0.020604
e_RAdeg,0.006865,0.197477,0.006954,-0.012804,0.085810,0.005190,0.014549,1.000000,0.956556,0.644871,0.727361,0.705992,0.002019,-0.007308,0.007294,-0.024354,-0.005258,0.000518,-0.007871,-0.011045,0.000668,0.008691,0.175326,0.088454,0.141483,0.177911,0.171560,0.274465,0.000956,0.095318,0.005908,0.091494,0.191743,0.687127,0.141807,-0.104973,0.181119,0.202796,0.322105,0.036536,-0.002833,-0.057801,0.136553,-0.244403,0.028315,0.036904,0.005549,0.151275
e_DEdeg,0.006921,0.198944,0.006961,-0.016224,

In [6]:
df[objCols]

,Proxy,RAhms,DEdms,r_Vmag,AstroRef,m_BTmag,r_B-V,r_V-I,CombMag,m_Hpmag,HvarType,CCDM,n_CCDM,MultFlag,Qual,m_HIP,Survey,Chart,Notes,BD,CoD,CPD,SpType,r_SpType
0,NaN,00 00 00.91,-19 29 55.8,G,+,NaN,G,I,NaN,NaN,C,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,B-20 6688,NaN,NaN,K3V,4
1,NaN,00 00 01.20,+38 51 33.4,G,NaN,NaN,G,H,NaN,NaN,C,00000+3852,I,NaN,NaN,NaN,S,NaN,NaN,B+38 5108,NaN,NaN,B9,S
2,NaN,00 00 05.41,+20 02 11.8,G,NaN,NaN,G,H,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B+19 5185,NaN,NaN,G0,S
3,NaN,00 00 06.55,+25 53 11.3,H,NaN,NaN,T,O,NaN,NaN,V,NaN,NaN,NaN,S,NaN,NaN,G,P,B+25 5054,NaN,NaN,M6e-M8.5e Tc,G
4,NaN,00 00 08.48,+36 35 09.4,H,NaN,NaN,T,L,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B+35 5149,NaN,NaN,G5,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71617,NaN,23 59 49.40,+21 08 59.1,H,NaN,NaN,G,H,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B+20 5413,NaN,NaN,F0,S
71618,NaN,23 59 49.64,-37 51 28.6,H,NaN,NaN,T,L,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C-38 15704,P-38 8593,K0,S
71619,NaN,23 59 51.30,+11 40 25.4,H,NaN,NaN,T,L,NaN,NaN,V,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,B+10 5018,NaN,NaN,K2,S
71620,NaN,23 59 53.74,-22 25 41.4,G,NaN,NaN,G,H,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,C-23 18108,P-23 8366,G2V,4


In [7]:
df[numCols]

,HIP,Vmag,RAdeg,DEdeg,Plx,pmRA,pmDE,e_RAdeg,e_DEdeg,e_Plx,e_pmRA,e_pmDE,DE:RA,Plx:RA,Plx:DE,pmRA:RA,pmRA:DE,pmRA:Plx,pmDE:RA,pmDE:DE,pmDE:Plx,pmDE:pmRA,F1,F2,BTmag,e_BTmag,VTmag,e_VTmag,B-V,e_B-V,V-I,e_V-I,Hpmag,e_Hpmag,Hpscat,o_Hpmag,Hpmax,HPmin,Nsys,Ncomp,theta,rho,e_rho,dHp,e_dHp,HD,(V-I)red,Target
0,2,9.27,0.003797,-19.498837,21.90,181.21,-0.93,1.28,0.70,3.10,1.74,0.92,0.12,-0.14,-0.24,-0.29,0.01,0.21,-0.02,-0.19,-0.28,0.14,2.0,1.45,10.519,0.033,9.378,0.021,0.999,0.002,1.04,0.00,9.4017,0.0017,0.015,120.0,9.37,9.44,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224690.0,1.04,0
1,3,6.61,0.005008,38.859286,2.81,5.24,-2.91,0.53,0.40,0.63,0.57,0.47,0.06,0.09,0.04,0.43,-0.01,-0.06,0.03,0.24,0.07,0.21,0.0,-0.45,6.576,0.004,6.621,0.005,-0.019,0.004,0.00,0.00,6.6081,0.0007,0.008,127.0,6.60,6.62,1.0,1.0,NaN,NaN,NaN,NaN,NaN,224699.0,0.00,0
2,7,9.64,0.022549,20.036602,17.74,-208.12,-200.79,1.01,0.79,1.30,1.13,0.82,0.32,0.08,-0.02,-0.04,0.12,0.06,0.11,0.00,0.16,0.43,0.0,0.21,10.542,0.039,9.679,0.030,0.740,0.020,0.79,0.02,9.6795,0.0021,0.017,104.0,9.65,9.72,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.79,0
3,8,9.05,0.027292,25.886474,5.17,19.09,-5.66,1.70,0.93,1.95,1.54,0.88,0.27,-0.66,-0.36,-0.38,-0.12,0.36,-0.21,-0.24,0.32,0.18,0.0,0.98,10.433,0.055,9.151,0.029,1.102,0.051,3.92,0.39,8.5522,0.1671,1.460,77.0,7.15,11.25,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224709.0,4.82,1
4,9,8.59,0.035342,36.585938,4.81,-6.30,8.42,0.86,0.55,0.99,1.02,0.65,0.03,0.16,-0.01,0.00,0.07,-0.02,0.08,0.04,0.10,0.13,3.0,-1.26,9.962,0.025,8.711,0.015,1.067,0.023,1.03,0.02,8.7534,0.0018,0.014,107.0,8.73,8.78,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224708.0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71617,118313,8.94,359.955817,21.149740,2.32,0.28,-15.77,0.89,0.70,1.15,0.90,0.70,0.28,0.21,0.03,0.02,-0.03,0.06,0.03,-0.25,0.30,0.27,0.0,1.21,9.400,0.019,8.989,0.019,0.423,0.015,0.49,0.02,9.0336,0.0021,0.013,92.0,9.01,9.06,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224671.0,0.49,0
71618,118314,9.27,359.956824,-37.857956,0.93,-6.60,-4.60,1.03,0.76,1.32,1.35,0.85,0.11,-0.10,-0.05,-0.21,-0.03,0.23,-0.06,-0.38,0.00,-0.13,0.0,0.26,10.731,0.034,9.378,0.018,1.147,0.030,1.11,0.03,9.4350,0.0020,0.022,148.0,9.38,9.47,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.03,0
71619,118318,6.99,359.963744,11.673709,1.92,-2.16,2.09,0.78,0.50,0.91,0.88,0.60,0.33,-0.02,-0.09,-0.19,-0.03,0.21,-0.02,-0.11,0.27,0.49,0.0,2.00,9.107,0.015,7.186,0.007,1.595,0.013,1.61,0.01,7.1043,0.0018,0.014,92.0,7.08,7.12,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224673.0,1.56,1
71620,118319,8.23,359.973913,-22.428180,10.63,148.74,27.53,0.97,0.58,1.17,1.10,0.52,0.04,0.17,0.04,-0.11,0.10,0.17,0.07,-0.31,0.12,-0.05,0.0,1.19,9.019,0.013,8.295,0.011,0.639,0.015,0.70,0.01,8.3699,0.0014,0.016,118.0,8.34,8.39,NaN,1.0,NaN,NaN,NaN,NaN,NaN,224693.0,0.70,0


In [8]:
for col in numCols:
    print(col)
    display(df[col].value_counts())
    display(df[col].isna().sum() / len(df))

HIP


2047      1
80242     1
98661     1
104806    1
117100    1
         ..
44287     1
48381     1
46332     1
34042     1
2049      1
Name: HIP, Length: 71622, dtype: int64

0.0

Vmag


8.65     300
8.94     299
8.27     299
8.44     297
8.40     294
        ... 
13.31      1
3.87       1
12.86      1
1.83       1
1.67       1
Name: Vmag, Length: 1085, dtype: int64

...

0.64    976
0.65    958
0.58    922
0.57    907
0.98    862
       ... 
6.99      1
3.94      1
6.10      1
4.09      1
3.89      1
Name: (V-I)red, Length: 559, dtype: int64

0.0

Target


0    46552
1    25070
Name: Target, dtype: int64

0.0

In [9]:
df[df['theta'].isna()]['rho'].isna().sum()

61778

In [10]:
df['r_SpType'].value_counts()

S    35184
X     8494
3     6436
1     6403
4     6226
2     5279
G     1132
K      653
Name: r_SpType, dtype: int64

In [11]:
df

,HIP,Proxy,RAhms,DEdms,Vmag,r_Vmag,RAdeg,DEdeg,AstroRef,Plx,pmRA,pmDE,e_RAdeg,e_DEdeg,e_Plx,e_pmRA,e_pmDE,DE:RA,Plx:RA,Plx:DE,pmRA:RA,pmRA:DE,pmRA:Plx,pmDE:RA,pmDE:DE,pmDE:Plx,pmDE:pmRA,F1,F2,BTmag,e_BTmag,VTmag,e_VTmag,m_BTmag,B-V,e_B-V,r_B-V,V-I,e_V-I,r_V-I,CombMag,Hpmag,e_Hpmag,Hpscat,o_Hpmag,m_Hpmag,Hpmax,HPmin,HvarType,CCDM,n_CCDM,Nsys,Ncomp,MultFlag,Qual,m_HIP,theta,rho,e_rho,dHp,e_dHp,Survey,Chart,Notes,HD,BD,CoD,CPD,(V-I)red,SpType,r_SpType,Target
0,2,NaN,00 00 00.91,-19 29 55.8,9.27,G,0.003797,-19.498837,+,21.90,181.21,-0.93,1.28,0.70,3.10,1.74,0.92,0.12,-0.14,-0.24,-0.29,0.01,0.21,-0.02,-0.19,-0.28,0.14,2.0,1.45,10.519,0.033,9.378,0.021,NaN,0.999,0.002,G,1.04,0.00,I,NaN,9.4017,0.0017,0.015,120.0,NaN,9.37,9.44,C,NaN,NaN,NaN,1.0,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224690.0,B-20 6688,NaN,NaN,1.04,K3V,4,0
1,3,NaN,00 00 01.20,+38 51 33.4,6.61,G,0.005008,38.859286,NaN,2.81,5.24,-2.91,0.53,0.40,0.63,0.57,0.47,0.06,0.09,0.04,0.43,-0.01,-0.06,0.03,0.24,0.07,0.21,0.0,-0.45,6.576,0.004,6.621,0.005,NaN,-0.019,0.004,G,0.00,0.00,H,NaN,6.6081,0.0007,0.008,127.0,NaN,6.60,6.62,C,00000+3852,I,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,224699.0,B+38 5108,NaN,NaN,0.00,B9,S,0
2,7,NaN,00 00 05.41,+20 02 11.8,9.64,G,0.022549,20.036602,NaN,17.74,-208.12,-200.79,1.01,0.79,1.30,1.13,0.82,0.32,0.08,-0.02,-0.04,0.12,0.06,0.11,0.00,0.16,0.43,0.0,0.21,10.542,0.039,9.679,0.030,NaN,0.740,0.020,G,0.79,0.02,H,NaN,9.6795,0.0021,0.017,104.0,NaN,9.65,9.72,C,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B+19 5185,NaN,NaN,0.79,G0,S,0
3,8,NaN,00 00 06.55,+25 53 11.3,9.05,H,0.027292,25.886474,NaN,5.17,19.09,-5.66,1.70,0.93,1.95,1.54,0.88,0.27,-0.66,-0.36,-0.38,-0.12,0.36,-0.21,-0.24,0.32,0.18,0.0,0.98,10.433,0.055,9.151,0.029,NaN,1.102,0.051,T,3.92,0.39,O,NaN,8.5522,0.1671,1.460,77.0,NaN,7.15,11.25,V,NaN,NaN,NaN,1.0,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,P,224709.0,B+25 5054,NaN,NaN,4.82,M6e-M8.5e Tc,G,1
4,9,NaN,00 00 08.48,+36 35 09.4,8.59,H,0.035342,36.585938,NaN,4.81,-6.30,8.42,0.86,0.55,0.99,1.02,0.65,0.03,0.16,-0.01,0.00,0.07,-0.02,0.08,0.04,0.10,0.13,3.0,-1.26,9.962,0.025,8.711,0.015,NaN,1.067,0.023,T,1.03,0.02,L,NaN,8.7534,0.0018,0.014,107.0,NaN,8.73,8.78,C,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224708.0,B+35 5149,NaN,NaN,1.00,G5,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71617,118313,NaN,23 59 49.40,+21 08 59.1,8.94,H,359.955817,21.149740,NaN,2.32,0.28,-15.77,0.89,0.70,1.15,0.90,0.70,0.28,0.21,0.03,0.02,-0.03,0.06,0.03,-0.25,0.30,0.27,0.0,1.21,9.400,0.019,8.989,0.019,NaN,0.423,0.015,G,0.49,0.02,H,NaN,9.0336,0.0021,0.013,92.0,NaN,9.01,9.06,C,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224671.0,B+20 5413,NaN,NaN,0.49,F0,S,0
71618,118314,NaN,23 59 49.64,-37 51 28.6,9.27,H,359.956824,-37.857956,NaN,0.93,-6.60,-4.60,1.03,0.76,1.32,1.35,0.85,0.11,-0.10,-0.05,-0.21,-0.03,0.23,-0.06,-0.38,0.00,-0.13,0.0,0.26,10.731,0.034,9.378,0.018,NaN,1.147,0.030,T,1.11,0.03,L,NaN,9.4350,0.0020,0.022,148.0,NaN,9.38,9.47,C,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C-38 15704,P-38 8593,1.03,K0,S,0
71619,118318,NaN,23 59 51.30,+11 40 25.4,6.99,H,359.963744,11.673709,NaN,1.92,-2.16,2.09,0.78,0.50,0.91,0.88,0.60,0.33,-0.02,-0.09,-0.19,-0.03,0.21,-0.02,-0.11,0.27,0.49,0.0,2.00,9.107,0.015,7.186,0.007,NaN,1.595,0.013,T,1.61,0.01,L,NaN,7.1043,0.0018,0.014,92.0,NaN,7.08,7.12,V,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,224673.0,B+10 5018,NaN,NaN,1.56,K2,S,1
71620,118319,NaN,23 59 53.74,-22 25 41.4,8.23,G,359.973913,-22.428180,NaN,10.63,148.74,27.53,0.97,0.58,1.17,1.10,0.52,0.04,0.17,0.04,-0.11,0.10,0.17,0.07,-0.31,0.12,-0.05,0.0,1.19,9.019,0.013,8.295,0.011,NaN,0.639,0.015,G,0.70,0.01,H,NaN,8.3699,0.0014,0.016,118.0,NaN,8.34,8.3

In [12]:
for col in df.columns:
    if 'T' in df[col].value_counts().index:
        print(col)

Proxy
r_Vmag
r_B-V
r_V-I


In [189]:
mode_cols = ['r_B-V','r_V-I','r_Vmag']
mode_imp = Pipeline(steps=[
    ('mode_imp', SimpleImputer(strategy='most_frequent')),
    ('mode_enc', OneHotEncoder(handle_unknown='ignore'))
])

const_cols = ['AstroRef','m_BTmag','CombMag','Proxy','m_Hpmag','CCDM','n_CCDM','MultFlag','Qual','Survey',
              'Chart','Notes','r_SpType']
const_imp = Pipeline(steps=[
    ('const_imp', SimpleImputer(strategy='constant',fill_value='Null')),
    ('const_enc', OneHotEncoder(handle_unknown='ignore'))
])

num_cols = ['Vmag', "RAdeg", 'DEdeg', 'Plx', 'pmRA', 'pmDE', 'e_RAdeg', 'e_DEdeg', 'e_Plx', 'e_pmRA',
           'e_pmDE', 'DE:RA', 'Plx:RA', 'Plx:DE', 'pmRA:RA', 'pmRA:DE', 'pmRA:Plx', 'pmDE:RA', 'pmDE:DE',
           'pmDE:Plx', 'pmDE:pmRA', 'F1', 'F2', 'BTmag', 'VTmag', 'e_VTmag', 'B-V', 'e_B-V', 'V-I', 'e_V-I',
           'Hpmag', 'e_Hpmag', 'Hpscat', 'o_Hpmag', 'Hpmax', 'HPmin', 'rho', 'e_rho', 'dHp', 'e_dHp', '(V-I)red']
num_imp = Pipeline(steps=[
    ('num_imp',SimpleImputer(strategy='mean'))
])

missing_ind_cols = ['theta']
missing_ind_imp = Pipeline(steps=[
    ('missing_ind_imp', SimpleImputer(strategy='mean', add_indicator=True))
])

num_const_cols = ['Nsys', 'Ncomp']
num_const_imp = Pipeline(steps=[
    ('num_const_imp', SimpleImputer(strategy='constant', fill_value=1.0))
])

use_cols = [
#     *mode_cols,
#     *const_cols,
    *num_cols, 
#     *missing_ind_cols, 
    *num_const_cols
]

In [174]:
preprocessor = ColumnTransformer(transformers=[
#     ('mode_pre', mode_imp, mode_cols),
#     ('const_pre', const_imp, const_cols),
    ('num_pre', num_imp, num_cols),
#     ('missing_ind_pre', missing_ind_imp, missing_ind_cols),
    ('num_const_pre', num_const_imp, num_const_cols)
])

In [175]:
# tree = Pipeline(steps=[
#     ('preprocessor',preprocessor),
# #     ('scaler'`, StandardScaler(with_mean=False)),
#     ('tree', DecisionTreeClassifier(random_state=42, max_depth=50, min_samples_split=100))
# ])
x_tr_pro = preprocessor.fit_transform(x_train[use_cols])
tree = DecisionTreeClassifier(random_state=42, max_depth=50, min_samples_split=100)

selector = RFE(tree, n_features_to_select=3, step=1)
selector.fit(x_tr_pro, y_train)

ranking = selector.ranking_

# param_grid = {
#     "tree__max_depth" : [1, 10, 50, 100],
#     "tree__min_samples_split" : [None, 5, 10, 50, 100],
# }
# cv_rfc = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5)
# cv_rfc.fit(x_train[use_cols], y_train)

# print(cv_rfc.best_score_)

# estimator = cv_rfc.best_estimator_
# print(estimator)
# displayMetrics(estimator, x_train[use_cols], x_test[use_cols], y_train, y_test)

In [176]:
ranking

array([16, 21, 18, 19, 34, 23, 24,  8, 17, 31, 20, 37, 25, 22, 29, 30, 33,
       35, 40, 28, 38, 32, 15, 10,  5,  4,  2, 13, 26, 39,  7,  3,  1, 11,
        1, 36, 41,  9, 27, 12, 14,  6,  1])

In [190]:
# mode_pre_col = preprocessor.named_transformers_["mode_pre"].get_feature_names()
# const_pre_col = preprocessor.named_transformers_["const_pre"].get_feature_names()

grand_col = [
#     *mode_pre_col, 
#     *const_pre_col, 
    *num_cols, 
#     *missing_ind_cols, 
    *num_const_cols
]

In [191]:
grand_col

['Vmag',
 'RAdeg',
 'DEdeg',
 'Plx',
 'pmRA',
 'pmDE',
 'e_RAdeg',
 'e_DEdeg',
 'e_Plx',
 'e_pmRA',
 'e_pmDE',
 'DE:RA',
 'Plx:RA',
 'Plx:DE',
 'pmRA:RA',
 'pmRA:DE',
 'pmRA:Plx',
 'pmDE:RA',
 'pmDE:DE',
 'pmDE:Plx',
 'pmDE:pmRA',
 'F1',
 'F2',
 'BTmag',
 'VTmag',
 'e_VTmag',
 'B-V',
 'e_B-V',
 'V-I',
 'e_V-I',
 'Hpmag',
 'e_Hpmag',
 'Hpscat',
 'o_Hpmag',
 'Hpmax',
 'HPmin',
 'rho',
 'e_rho',
 'dHp',
 'e_dHp',
 '(V-I)red',
 'Nsys',
 'Ncomp']

In [192]:
# grand_obj = dict(zip(grand_col, ranking))

In [197]:
grand_num = dict(zip(grand_col, ranking))

In [198]:
grand_num

{'Vmag': 16,
 'RAdeg': 21,
 'DEdeg': 18,
 'Plx': 19,
 'pmRA': 34,
 'pmDE': 23,
 'e_RAdeg': 24,
 'e_DEdeg': 8,
 'e_Plx': 17,
 'e_pmRA': 31,
 'e_pmDE': 20,
 'DE:RA': 37,
 'Plx:RA': 25,
 'Plx:DE': 22,
 'pmRA:RA': 29,
 'pmRA:DE': 30,
 'pmRA:Plx': 33,
 'pmDE:RA': 35,
 'pmDE:DE': 40,
 'pmDE:Plx': 28,
 'pmDE:pmRA': 38,
 'F1': 32,
 'F2': 15,
 'BTmag': 10,
 'VTmag': 5,
 'e_VTmag': 4,
 'B-V': 2,
 'e_B-V': 13,
 'V-I': 26,
 'e_V-I': 39,
 'Hpmag': 7,
 'e_Hpmag': 3,
 'Hpscat': 1,
 'o_Hpmag': 11,
 'Hpmax': 1,
 'HPmin': 36,
 'rho': 41,
 'e_rho': 9,
 'dHp': 27,
 'e_dHp': 12,
 '(V-I)red': 14,
 'Nsys': 6,
 'Ncomp': 1}

In [195]:
max(grand_num.values())
final_list = [
    "Ncomp",
    "Hpmax",
    "Nsys",
    "Hpscat",
    "e_Hpmag",
    'B-V',
    'e_VTmag',
    'VTmag'
]

41

In [196]:
x = 42
while x > 1:
    x -= 1
    grand_col1 = []
    grand_col1 = [keys for keys in grand_num.keys()]
    
    preprocessor = ColumnTransformer(transformers=[
        ('num_pre', num_imp, num_cols),
        ('num_const_pre', num_const_imp, num_const_cols)
    ])
    
    tree = Pipeline(steps=[
        ('preprocessor',preprocessor),
#       ('scaler', StandardScaler(with_mean=False)),
        ('tree', DecisionTreeClassifier(random_state=42, max_depth=15, min_samples_split=74))
    ])
    
    tree.fit(x_train[grand_col1], y_train)
    train_preds = tree.predict(x_train[grand_col1])
    test_preds = tree.predict(x_test[grand_col1])
    
    train_accuracy = accuracy_score(y_train, train_preds)
    test_accuracy = accuracy_score(y_test, test_preds)
    train_recall = recall_score(y_train, train_preds, zero_division=0)
    test_recall = recall_score(y_test, test_preds, zero_division=0)
    print(x, "train acc", train_accuracy, "test acc", test_accuracy, "train rec", train_recall, "test rec", test_recall)
    
    drop_col = max(grand_num, key=grand_num.get)
    num_cols.remove(drop_col)
    del grand_num[drop_col]

41 train acc 0.9398317074986968 test acc 0.9236010275885178 train rec 0.8671850037277665 test rec 0.8471074380165289
40 train acc 0.9398503239258321 test acc 0.9237127219926282 train rec 0.8668122270742358 test rec 0.8469485060394152
39 train acc 0.9398689403529674 test acc 0.923991958002904 train rec 0.8667589732665886 test rec 0.8475842339478703
38 train acc 0.9398130910715615 test acc 0.9240478052049592 train rec 0.8656938971136436 test rec 0.8467895740623014
37 train acc 0.9398317074986968 test acc 0.9237685691946833 train rec 0.8654808818830546 test rec 0.8459949141767323
36 train acc 0.9398503239258321 test acc 0.9238244163967385 train rec 0.8656406433059963 test rec 0.8464717101080738
35 train acc 0.9398503239258321 test acc 0.9239361108008489 train rec 0.8656406433059963 test rec 0.8466306420851876
34 train acc 0.9398875567801027 test acc 0.9238244163967385 train rec 0.8660666737671744 test rec 0.8467895740623014
33 train acc 0.939906173207238 test acc 0.9239361108008489 train 

ValueError: list.remove(x): x not in list